In [1]:
import sys
sys.path.append('..\.')
from utils.qSLP import qSLP
from utils.Utils import get_params, parity
from utils.import_data import get_dataset

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from qiskit.utils import QuantumInstance
from qiskit.algorithms.optimizers import COBYLA
from qiskit import Aer, QuantumCircuit
from qiskit.circuit.library import ZZFeatureMap, ZFeatureMap
from qiskit.circuit.library import RealAmplitudes
from qiskit_machine_learning.neural_networks import CircuitQNN
from qiskit_machine_learning.algorithms.classifiers import NeuralNetworkClassifier

from sklearn.metrics import accuracy_score

seed = 42
max_iter = 150

D:\conda\envs\tesi3\lib\site-packages\qiskit\aqua\__init__.py:86: DeprecationWarning: The package qiskit.aqua is deprecated. It was moved/refactored to qiskit-terra For more information see <https://github.com/Qiskit/qiskit-aqua/blob/main/README.md#migration-guide>
  warn_package('aqua', 'qiskit-terra')


In [2]:
def get_qNNC(version = 1):
    if version == 1:
        feature_map = ZZFeatureMap(feature_dimension=2,
                            reps=1, entanglement='linear')
        ansatz = RealAmplitudes(2, reps=1)

    elif version == 2:
        feature_map = ZFeatureMap(feature_dimension=2,
                                reps=1)
        ansatz = RealAmplitudes(2, reps=2)

    else:
        raise Exception("Version not implemented")
    
    return feature_map, ansatz


### Obtain the dataset
The dataset is processed through the use a PCA in order to use only two high descriptive features. 


In [3]:
# Change to change the dataset
dataset = "iris01"

X_train, X_test, Y_train, Y_test = get_dataset(dataset)

[0.90539269 0.07445563]
98.0% of total variance is explained by 2 principal components


### Set the optimizer and the quantum instance


In [4]:
optimizer = COBYLA(maxiter=max_iter, tol=0.01, disp=False)
qinstance = QuantumInstance(Aer.get_backend('aer_simulator'),seed_simulator=seed,seed_transpiler=seed, shots=1024)
qinstance.backend.set_option("seed_simulator", seed)


### The model 
Build the model with the chosen parameters.


In [31]:
feature_map = ZZFeatureMap(feature_dimension=2, entanglement='linear')
kernel = QuantumKernel(feature_map=feature_map, quantum_instance=qinstance)
qsvc = QSVC(quantum_kernel=kernel)


## Training


In [33]:
    qsvc = qsvc.fit(X_train, Y_train)

In [36]:
    train_score = qsvc.score(X_train, Y_train)
    test_score = qsvc.score(X_test, Y_test)

In [38]:
print(train_score)
print(test_score)

0.9
0.95
